In [1]:
# Install necessary packages
!pip install requests
!pip install tqdm

import requests
from tqdm import tqdm

# Install necessary packages
!pip install PyMuPDF

import fitz  # PyMuPDF
import os
from IPython.display import display
import ipywidgets as widgets

# List of PDF URLs
pdf_urls = [
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/summary-of-consultation.pdf?v=336045",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/proposed-codes-at-a-glance.pdf?v=336047",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/vol1-overview-scope-regulatory-approach.pdf?v=336050",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/vol2-identifying-services-children-are-using.pdf?v=336051",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/vol3-causes-impacts-of-harms-to-children.pdf?v=336052",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/vol4-assessing-risks-of-harms-to-children-online.pdf?v=336053",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/vol5-what-should-services-do-to-mitigate-risks.pdf?v=336054",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/a1-4-generic-annexes.pdf?v=336073",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/a5-draft-childrens-access-assessments-guidance.pdf?v=336055",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/a6-draft-childrens-risk-assessment-guidance-risk-profiles.pdf?v=368062",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/a7-draft-childrens-safety-code-user-to-user-services.pdf?v=336059",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/a8-draft-childrens-safety-code-search-services.pdf?v=336061",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/a9-proposes-addenda-to-illegal-codes.pdf?v=336066",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/a10-15-other-annexes.pdf?v=336071",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/large-services-guidance.pdf?v=336074",
    "https://www.ofcom.org.uk/siteassets/resources/documents/consultations/category-1-10-weeks/284469-consultation-protecting-children-from-harms-online/associated-documents/ofcoms-engagement-with-children-on-the-draft-childrens-safety-codes.pdf?v=368473"
]

def download_pdf(url):
    response = requests.get(url)
    if response.status_code == 200:
        file_path = url.split("/")[-1].split("?")[0]  # Handle URL parameters
        with open(file_path, 'wb') as f:
            f.write(response.content)
        return file_path
    else:
        print(f"Failed to download {url}")
        return None

def main():
    for pdf_url in tqdm(pdf_urls):
        download_pdf(pdf_url)

main()


100%|██████████| 16/16 [00:03<00:00,  4.44it/s]


In [9]:
# List of downloaded PDF filenames
pdf_filenames = [
    "summary-of-consultation.pdf",
    "proposed-codes-at-a-glance.pdf",
    "vol1-overview-scope-regulatory-approach.pdf",
    "vol2-identifying-services-children-are-using.pdf",
    "vol3-causes-impacts-of-harms-to-children.pdf",
    "vol4-assessing-risks-of-harms-to-children-online.pdf",
    "vol5-what-should-services-do-to-mitigate-risks.pdf",
    "a1-4-generic-annexes.pdf",
    "a5-draft-childrens-access-assessments-guidance.pdf",
    "a6-draft-childrens-risk-assessment-guidance-risk-profiles.pdf",
    "a7-draft-childrens-safety-code-user-to-user-services.pdf",
    "a8-draft-childrens-safety-code-search-services.pdf",
    "a9-proposes-addenda-to-illegal-codes.pdf",
    "a10-15-other-annexes.pdf",
    "large-services-guidance.pdf",
    "ofcoms-engagement-with-children-on-the-draft-childrens-safety-codes.pdf"
]

def search_pdf(file_path, search_word):
    doc = fitz.open(file_path)
    results = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text("text")
        search_word_lower = search_word.lower()
        start = 0
        while True:
            start = text.lower().find(search_word_lower, start)
            if start == -1:
                break
            before = max(start - 100, 0)
            after = min(start + len(search_word) + 100, len(text))
            snippet = text[before:after]
            results.append((file_path, page_num + 1, snippet))
            start += len(search_word)
    return results

def search_and_display_results(search_word):
    all_results = []
    for pdf_file in pdf_filenames:
        if os.path.exists(pdf_file):
            print()
            print(f"Searching in {pdf_file}...")
            print()
            results = search_pdf(pdf_file, search_word)
            if results:
                for result in results:
                    print(f"Found '{search_word}' in {result[0]} on page {result[1]}:\n{result[2]}\n")
                    print()
            else:
                print(f"'{search_word}' not found in {pdf_file}.")
                print()
        else:
            print(f"File {pdf_file} does not exist.")

    if not all_results:
        print(f"'{search_word}' not found in any document.")

# Create a form for user input
search_word_input = widgets.Text(
    value='',
    placeholder='Type something',
    description='Search word:',
    disabled=False
)

search_button = widgets.Button(
    description='Search',
    disabled=False,
    button_style='',
    tooltip='Click to search',
    icon='search'
)

def on_search_button_clicked(b):
    search_word = search_word_input.value
    search_and_display_results(search_word)

search_button.on_click(on_search_button_clicked)

display(search_word_input, search_button)


Text(value='', description='Search word:', placeholder='Type something')

Button(description='Search', icon='search', style=ButtonStyle(), tooltip='Click to search')


Searching in summary-of-consultation.pdf...

'existing measures' not found in summary-of-consultation.pdf.


Searching in proposed-codes-at-a-glance.pdf...

'existing measures' not found in proposed-codes-at-a-glance.pdf.


Searching in vol1-overview-scope-regulatory-approach.pdf...

'existing measures' not found in vol1-overview-scope-regulatory-approach.pdf.


Searching in vol2-identifying-services-children-are-using.pdf...

'existing measures' not found in vol2-identifying-services-children-are-using.pdf.


Searching in vol3-causes-impacts-of-harms-to-children.pdf...

'existing measures' not found in vol3-causes-impacts-of-harms-to-children.pdf.


Searching in vol4-assessing-risks-of-harms-to-children-online.pdf...

'existing measures' not found in vol4-assessing-risks-of-harms-to-children-online.pdf.


Searching in vol5-what-should-services-do-to-mitigate-risks.pdf...

Found 'existing measures' in vol5-what-should-services-do-to-mitigate-risks.pdf on page 26:
o a lack of robust ev